In [15]:
# NeuralNetwork class (original lecture note)

import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

%matplotlib inline

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

class NeuralNetwork:
    
    # 생성자
    def __init__(self, input_nodes, hidden_nodes, output_nodes):
        self.input_nodes = input_nodes # 784
        self.hidden_nodes = hidden_nodes # 100
        self.output_nodes = output_nodes # 10
        
        # 2층 hidden layer unit
        # 가중치 W, 바이어스 b 초기화
        self.W2 = np.random.rand(self.input_nodes, self.hidden_nodes) # W2 = (784x100)
        self.b2 = np.random.rand(self.hidden_nodes) # b2 = (100,)
        
        # 3층 output layer unit
        self.W3 = np.random.rand(self.hidden_nodes, self.output_nodes) # W3 = (100x10)
        self.b3 = np.random.rand(self.output_nodes) # (10,)
        
        # 학습률 learning rate 초기화
        self.learning_rate = 1e-5
        
    # feed forward 를 이용하여 입력층에서부터 출력층까지 데이터를 전달하고 손실함수값 계산
    # loss_val(self) 메서드와 동일한 코드. loss_val(self)은 외부 출력용으로 사용됨
    
    def feed_forward(self):
        delta = 1e-7
        
        z1 = np.dot(self.input_data, self.W2) + self.b2
        y1 = sigmoid(z1)
        
        z2 = np.dot(y1, self.W3) +self.b3
        y = sigmoid(z2)
        
        return -np.sum( self.target_data*np.log(y+delta)+(1-self.target_data)*np.log((1-y)+delta))
    
    # 손실 값 계산
    def loss_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z1 = np.dot(self.input_data, self.W2) + self.b2
        y1 = sigmoid(z1)
        
        z2 = np.dot(y1, self.W3) + self.b3
        y = sigmoid(z2)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # input_data : 784개, target_data : 10개
    def train(self, training_data):
        # normalize 
        self.target_data = np.zeros(output_nodes) + 0.01 # one-hot encoding 을 위한 10개 노드 0.01 초기화
        self.target_data[int(training_data[0])] = 0.99 # 정답을 나타내는 인덱스에 가장 큰 값인 0.99 초기화
        
        self.input_data = (training_data[1:]/255.0 *0.99) + 0.01 # 입력데이터는 0~255이기 때문에 overflow 발생 가능. 따라서 0~1 값으로 normalize
        
        f = lambda x : self.feed_forward()
        self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
        self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
        self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)
        
    def predict(self, input_data):
        z1 = np.dot(input_data, self.W2) + self.b2
        y1 = sigmoid(z1)
        
        z2 = np.dot(y1, self.W3) + self.b3
        y = sigmoid(z2)
        
        predicted_num = np.argmax(y) # 가장 큰 값을 가지는 인덱스를 정답으로 인식(argmax)
        
        return predicted_num
    
    def accuracy(self, test_data):
        matched_list = []
        not_matched_list = []
        
        for index in range(len(test_data)):
            label = int(test_data[index,0]) # test data의 1열에 있는 정답을 분리
            
            #normalize
            data = (test_data[index,1:] / 255.0 * 0.99) + 0.01 
            predicted_num = self.predict(data)
            if label == predicted_num:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        print("Current Accuracy = ", 100*(len(matched_list))/(len(test_data)), " %")
        return matched_list, not_matched_list

In [16]:
# training data 
training_data = np.loadtxt(r'C:\Users\allma\Desktop/mnist_train.csv', delimiter=',', dtype=np.float32)
# test data 
test_data = np.loadtxt(r'C:\Users\allma\Desktop/mnist_test.csv', delimiter=',', dtype=np.float32)

print("training_data.shape = ", training_data.shape)
print("test_data.shape = ", test_data.shape)

input_nodes = 784
hidden_nodes = 100
output_nodes = 10

nn = NeuralNetwork(input_nodes, hidden_nodes, output_nodes)

for step in range(10): # 10개로 학습
    index = np.random.randint(0, len(training_data)-1) # 총 6만개의 데이터 가운데 랜덤하게 선택
    
    nn.train(training_data[index])
    
    print("step = ", step,", index = ", index, ", loss_val = ", nn.loss_val())

# 정확도 계산
nn.accuracy(test_data)

training_data.shape =  (60000, 785)
test_data.shape =  (10000, 785)
step =  0 , index =  34741 , loss_val =  143.77341309854825
step =  1 , index =  27828 , loss_val =  143.77341309854825
step =  2 , index =  2092 , loss_val =  143.77341309854825
step =  3 , index =  54910 , loss_val =  143.77341309854825
step =  4 , index =  50606 , loss_val =  143.77341309854825
step =  5 , index =  6983 , loss_val =  143.77341309854825
step =  6 , index =  21512 , loss_val =  143.77341309854825
step =  7 , index =  42753 , loss_val =  143.77341309854825
step =  8 , index =  44951 , loss_val =  143.77341309854825
step =  9 , index =  53061 , loss_val =  143.77341309854825
Current Accuracy =  9.8  %


([3,
  10,
  13,
  25,
  28,
  55,
  69,
  71,
  101,
  126,
  136,
  148,
  157,
  183,
  188,
  192,
  194,
  215,
  246,
  269,
  271,
  294,
  296,
  297,
  305,
  311,
  312,
  324,
  327,
  346,
  365,
  372,
  380,
  407,
  424,
  440,
  441,
  443,
  459,
  485,
  490,
  498,
  525,
  545,
  546,
  552,
  561,
  564,
  567,
  581,
  586,
  592,
  597,
  608,
  611,
  620,
  642,
  644,
  656,
  661,
  701,
  704,
  713,
  717,
  721,
  723,
  742,
  750,
  763,
  794,
  804,
  821,
  845,
  851,
  854,
  871,
  884,
  895,
  896,
  904,
  932,
  972,
  981,
  993,
  997,
  1001,
  1009,
  1034,
  1047,
  1061,
  1084,
  1094,
  1121,
  1148,
  1154,
  1176,
  1188,
  1191,
  1195,
  1197,
  1203,
  1218,
  1220,
  1223,
  1231,
  1246,
  1261,
  1271,
  1273,
  1288,
  1294,
  1297,
  1333,
  1379,
  1390,
  1399,
  1404,
  1408,
  1411,
  1416,
  1438,
  1445,
  1454,
  1457,
  1468,
  1487,
  1504,
  1517,
  1526,
  1533,
  1534,
  1535,
  1558,
  1565,
  1570,
  1589,
  1590